In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense,Activation
from keras.layers.recurrent import LSTM
from keras.layers import CuDNNLSTM
from keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from sklearn import preprocessing
import matplotlib.pyplot as plt

data=pd.read_csv(r"data4.csv",index_col='no')

internal=100

scalerx = preprocessing.MinMaxScaler().fit(data.iloc[:,1:])
data1=scalerx.transform(data.iloc[:,1:])

x=[]
y_former=[]
y=[]

for i in range(data1.shape[0]-internal-61):
    x.append(np.array(data1[i:i+internal]))
    y.append(np.array(data.iloc[i+internal+1:i+internal+61,1]))

x=np.array(x)
#y_former=np.array(y_former).reshape((-1,1))
y=np.array(y).reshape((-1,60))

# 归一化y
scalery = preprocessing.MinMaxScaler().fit(y)
y=scalery.transform(y)
#y_former=scalery.transform(y_former)

x_train,y_train=x[:15000],y[:15000]
x_test,y_test=x[15000:],y[15000:]

print(x.shape)
y.shape

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(17084, 100, 5)


(17084, 60)

In [3]:
model = Sequential()

model.add(CuDNNLSTM(32,input_shape=(x_train.shape[1], x_train.shape[2])))#, init='uniform'

model.add(Dense(60))
model.add(Activation('sigmoid'))
adam = Adam(lr=0.01)

model.compile(loss='mean_squared_error', optimizer=adam, metrics=["accuracy"])

hist = model.fit(x_train, y_train, batch_size=5, epochs=10, shuffle=True,verbose=1,validation_split=0.2)


y_predict = model.predict(x_test, batch_size=1, verbose=1)

y_predict = scalery.inverse_transform(y_predict)
y_test_yuan = scalery.inverse_transform(y_test)


y_train_predict = model.predict(x_train, batch_size=1, verbose=1)

y_train_predict = scalery.inverse_transform(y_train_predict)
y_train_yuan = scalery.inverse_transform(y_train)

Train on 12000 samples, validate on 3000 samples
Epoch 1/10
12000/12000 [==============================] - 31s 3ms/step - loss: 0.0041 - acc: 0.0200 - val_loss: 0.0011 - val_acc: 0.0157
Epoch 2/10
12000/12000 [==============================] - 26s 2ms/step - loss: 0.0028 - acc: 0.0241 - val_loss: 0.0016 - val_acc: 0.0247
Epoch 3/10
12000/12000 [==============================] - 26s 2ms/step - loss: 0.0027 - acc: 0.0248 - val_loss: 0.0012 - val_acc: 0.0383
Epoch 4/10
12000/12000 [==============================] - 26s 2ms/step - loss: 0.0025 - acc: 0.0303 - val_loss: 0.0010 - val_acc: 0.0200
Epoch 5/10
12000/12000 [==============================] - 26s 2ms/step - loss: 0.0024 - acc: 0.0313 - val_loss: 0.0011 - val_acc: 0.0223
Epoch 6/10
12000/12000 [==============================] - 27s 2ms/step - loss: 0.0023 - acc: 0.0335 - val_loss: 0.0013 - val_acc: 0.0220
Epoch 7/10
12000/12000 [==============================] - 27s 2ms/step - loss: 0.0022 - acc: 0.0357 - val_loss: 0.0012 - val_acc:

In [4]:
print(y_predict.shape)
print(y_test_yuan.shape)

print(y_train_yuan.shape)
print(y_train_predict.shape)

(2084, 60)
(2084, 60)
(15000, 60)
(15000, 60)


In [5]:
y_predict.reshape((-1,))

array([28.61728 , 28.619196, 28.769655, ..., 37.258167, 37.427547,
       37.290665], dtype=float32)

In [8]:
print(np.sqrt(sum((y_predict.reshape((-1,))-y_test_yuan.reshape((-1,)))**2)/len(y_predict.reshape((-1,)))))

print(np.sqrt(sum((y_train_yuan.reshape((-1,))-y_train_predict.reshape((-1,)))**2)/len(y_train_yuan.reshape((-1,)))))

7.3152831379899474
5.0017975998625985


### LSTM 循环

In [13]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense,Activation
from keras.layers.recurrent import LSTM
from keras.layers import CuDNNLSTM
from keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from sklearn import preprocessing
import matplotlib.pyplot as plt

data=pd.read_csv(r"data4.csv",index_col='no')

#internal=100
internals=[240]
lrs=[0.01]

scalerx = preprocessing.MinMaxScaler().fit(data.iloc[:,1:])
data1=scalerx.transform(data.iloc[:,1:])

rmse_train=[]
rmse_test=[]

for internal in internals:
    for lr in lrs:

        x=[]
        y_former=[]
        y=[]

        for i in range(data1.shape[0]-internal-31):
            x.append(np.array(data1[i:i+internal]))
            y.append(np.array(data.iloc[i+internal+1:i+internal+31,1]))

        x=np.array(x)
        #y_former=np.array(y_former).reshape((-1,1))
        y=np.array(y).reshape((-1,30))

        # 归一化y
        scalery = preprocessing.MinMaxScaler().fit(y)
        y=scalery.transform(y)
        #y_former=scalery.transform(y_former)

        x_train,y_train=x[:15000],y[:15000]
        x_test,y_test=x[15000:],y[15000:]

        model = Sequential()

        model.add(CuDNNLSTM(32,input_shape=(x_train.shape[1], x_train.shape[2])))#, init='uniform'

        model.add(Dense(30))
        model.add(Activation('sigmoid'))
        adam = Adam(lr=lr)

        model.compile(loss='mean_squared_error', optimizer=adam, metrics=["accuracy"])

        hist = model.fit(x_train, y_train, batch_size=5, epochs=20, shuffle=True,verbose=1,validation_split=0.2)


        y_predict = model.predict(x_test, batch_size=1, verbose=1)

        y_predict = scalery.inverse_transform(y_predict)
        y_test_yuan = scalery.inverse_transform(y_test)


        y_train_predict = model.predict(x_train, batch_size=1, verbose=1)

        y_train_predict = scalery.inverse_transform(y_train_predict)
        y_train_yuan = scalery.inverse_transform(y_train)

        rmse1=np.sqrt(sum((y_predict.reshape((-1,))-y_test_yuan.reshape((-1,)))**2)/len(y_predict.reshape((-1,))))
        rmse2=np.sqrt(sum((y_train_yuan.reshape((-1,))-y_train_predict.reshape((-1,)))**2)/len(y_train_yuan.reshape((-1,))))
        
        print([internal,lr,rmse1,rmse2])
        rmse_test.append(rmse1)
        rmse_train.append(rmse2)



Train on 12000 samples, validate on 3000 samples
Epoch 1/20
12000/12000 [==============================] - 42s 3ms/step - loss: 0.0024 - acc: 0.0386 - val_loss: 0.0016 - val_acc: 0.0307
Epoch 2/20
12000/12000 [==============================] - 40s 3ms/step - loss: 0.0018 - acc: 0.0435 - val_loss: 0.0010 - val_acc: 0.0283
Epoch 3/20
12000/12000 [==============================] - 40s 3ms/step - loss: 0.0017 - acc: 0.0423 - val_loss: 9.7933e-04 - val_acc: 0.0360
Epoch 4/20
12000/12000 [==============================] - 40s 3ms/step - loss: 0.0017 - acc: 0.0460 - val_loss: 9.8070e-04 - val_acc: 0.0560
Epoch 5/20
12000/12000 [==============================] - 39s 3ms/step - loss: 0.0016 - acc: 0.0450 - val_loss: 9.8775e-04 - val_acc: 0.0333
Epoch 6/20
12000/12000 [==============================] - 39s 3ms/step - loss: 0.0016 - acc: 0.0479 - val_loss: 9.8414e-04 - val_acc: 0.0413
Epoch 7/20
12000/12000 [==============================] - 39s 3ms/step - loss: 0.0015 - acc: 0.0500 - val_loss: 0

In [14]:
print(rmse_train)
print(rmse_test)

[4.49163394522702]
[4.1180641534045614]


In [15]:
np.save("data/y_test.npy",y_test_yuan)
np.save("data/y_test_pre.npy",y_predict)
np.save("data/y_train.npy",y_train_yuan)
np.save("data/y_train_pre.npy",y_train_predict)


In [16]:
model.save('LSTM_model.h5')

In [24]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense,Activation
from keras.layers.recurrent import LSTM
from keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from sklearn import preprocessing
import matplotlib.pyplot as plt

data=pd.read_csv(r"data4.csv",index_col='no')

#internal=100
internals=[240]
lrs=[0.01]

scalerx = preprocessing.MinMaxScaler().fit(data.iloc[:,1:])
data1=scalerx.transform(data.iloc[:,1:])

rmse_train=[]
rmse_test=[]

for internal in internals:
    for lr in lrs:

        x=[]
        y_former=[]
        y=[]

        for i in range(data1.shape[0]-internal-31):
            x.append(np.array(data1[i:i+internal]))
            y.append(np.array(data.iloc[i+internal+1:i+internal+31,1]))

        x=np.array(x)
        #y_former=np.array(y_former).reshape((-1,1))
        y=np.array(y).reshape((-1,30))

        # 归一化y
        scalery = preprocessing.MinMaxScaler().fit(y)
        y=scalery.transform(y)
        #y_former=scalery.transform(y_former)

        x_train,y_train=x[:15000],y[:15000]
        x_test,y_test=x[15000:],y[15000:]

        model = Sequential()

        model.add(LSTM(32,input_shape=(x_train.shape[1], x_train.shape[2])))#, init='uniform'

        model.add(Dense(30))
        model.add(Activation('sigmoid'))
        adam = Adam(lr=lr)

        model.compile(loss='mean_squared_error', optimizer=adam, metrics=["accuracy"])

        hist = model.fit(x_train, y_train, batch_size=5, epochs=20, shuffle=True,verbose=1,validation_split=0.2)


        y_predict = model.predict(x_test, batch_size=1, verbose=1)

        y_predict = scalery.inverse_transform(y_predict)
        y_test_yuan = scalery.inverse_transform(y_test)


        y_train_predict = model.predict(x_train, batch_size=1, verbose=1)

        y_train_predict = scalery.inverse_transform(y_train_predict)
        y_train_yuan = scalery.inverse_transform(y_train)

        rmse1=np.sqrt(sum((y_predict.reshape((-1,))-y_test_yuan.reshape((-1,)))**2)/len(y_predict.reshape((-1,))))
        rmse2=np.sqrt(sum((y_train_yuan.reshape((-1,))-y_train_predict.reshape((-1,)))**2)/len(y_train_yuan.reshape((-1,))))
        
        print([internal,lr,rmse1,rmse2])
        rmse_test.append(rmse1)
        rmse_train.append(rmse2)



Train on 12000 samples, validate on 3000 samples
Epoch 1/20
  615/12000 [>.............................] - ETA: 14:07 - loss: 0.0169 - acc: 0.0325

KeyboardInterrupt: 